In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import os
import random
from scipy.misc import imsave, imresize
import warnings
import tensorflow as tf
from tensorflow import keras
import random



C:\Users\yangt\Anaconda3\Ana\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#get training and test data
df_training = pd.read_csv('training_set.csv')
df_test = pd.read_csv('test_set.csv')

In [3]:
#read from csv and get aggression rating
def readToDict(data):
    useful_data = {}
    for player in data.itertuples(index = True, name='Pandas'):
    #print()
        details = []
        p_id = getattr(player,"ID")
        name = getattr(player,"Name")
        image_id = str(p_id) +'.jpg'
    
        aggression = getattr(player,"Aggression")
        #rating = getattr(player,"Overall")
        details.append(name)
        details.append(aggression)
        details.append(image_id)
        
        #details.append(rating)
        useful_data[p_id] = details
    return useful_data

training_dict = readToDict(df_training)
test_dict = readToDict(df_test)
print(len(training_dict))
print(len(test_dict))

12076
3020


In [4]:
def one_hot_encoding(aggression):
    rating = int(aggression)
    
    if rating <= 59:
        one_hot = np.array([1,0])
    else: one_hot = np.array([0,1])
    return one_hot
one_hot_encoding(59)

array([1, 0])

In [5]:
def get_training_data(training_dict):
    train_images = []
    for key in training_dict:
        #print(key)
        n_path = os.path.join('faces',training_dict[key][-1])
        img = mpimg.imread(n_path)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            img = imresize(img, (48, 48))
        
        #print(img.shape)
        train_images.append([np.array(img),one_hot_encoding(training_dict[key][1])])
        
    return train_images

training_data = get_training_data(training_dict)
test_data = get_training_data(test_dict)


        

In [6]:
#split into training and label

X_train = np.array([i[0] for i in training_data])
Y_train = np.array([i[1] for i in training_data])
print(X_train.shape)
print(Y_train.shape)
X_test = np.array([i[0] for i in test_data])
Y_test = np.array([i[1] for i in test_data])

(12076, 48, 48, 4)
(12076, 2)


In [7]:
def buildConvNet(x,y,eps = 5, dropout = True, dropRate = 0.2):
    #TODO: Implement a CNN here. dropout option is required.
    model = tf.keras.Sequential()
    lossType = tf.keras.losses.categorical_crossentropy
    opt = tf.keras.optimizers.Adam(lr = 1e-3)
    i_shape = (48,48,4)
    #add CNN layers
    model.add(tf.keras.layers.Conv2D(32,kernel_size = 5,padding = "same",activation = tf.nn.relu, input_shape = i_shape))
    model.add(tf.keras.layers.Conv2D(32,kernel_size = 5,padding = "same", activation = tf.nn.relu))
    model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
    
    
    
    model.add(tf.keras.layers.Conv2D(64,kernel_size = 5,padding = "same",activation = tf.nn.relu))
    model.add(tf.keras.layers.Conv2D(64,kernel_size = 5,padding = "same",activation = tf.nn.relu))
    model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
    
    
    
    model.add(tf.keras.layers.Conv2D(128,kernel_size = 5,padding = "same",activation = tf.nn.relu))
    model.add(tf.keras.layers.Conv2D(128,kernel_size = 5,padding = "same",activation = tf.nn.relu))
    model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
    
    
    
    
    model.add(tf.keras.layers.Dropout(dropRate))
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(512,activation = tf.nn.relu))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2,activation = tf.nn.softmax))
    model.compile(optimizer = opt, loss = lossType)
    model.fit(x,y,batch_size = 128, epochs = eps)
    model.summary()
    
    #model_json = model.to_json()
    return model

def train_network(xTrain,yTrain):
    
    return buildConvNet(xTrain, yTrain)

In [8]:
def runModel(model,y_param):
    preds = model.predict(y_param)
    for i in range(preds.shape[0]):
        oneHot = [0] * 2
        oneHot[np.argmax(preds[i])] = 1
        preds[i] = oneHot
    return preds

In [9]:
def evaluation(preds,y_label):
    acc = 0
    for i in range(preds.shape[0]):
        if np.array_equal(preds[i],y_label[i]): acc = acc+1
    accuracy = acc / preds.shape[0]
    #print("Classifier algorithm: %s" % ALGORITHM)
    print("Classifier accuracy: %f%%" % (accuracy * 100))
    print()
    return accuracy



In [10]:
model = train_network(X_train,Y_train)
preds = runModel(model,X_test)


Epoch 1/5
12076/12076 [==============================] - 416s 34ms/step - loss: 8.7310
Epoch 2/5
12076/12076 [==============================] - 405s 34ms/step - loss: 8.7624
Epoch 3/5
12076/12076 [==============================] - 405s 34ms/step - loss: 8.7624
Epoch 4/5
12076/12076 [==============================] - 400s 33ms/step - loss: 8.7624
Epoch 5/5
12076/12076 [==============================] - 403s 33ms/step - loss: 8.7624
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        3232      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2

In [11]:
accuracy = evaluation(preds,Y_test)

Classifier accuracy: 65.562914%



array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)